In [ ]:
#test

In [74]:
import json
import zeit
import configparser
import pandas as pd
import requests
import plotly.express as px

import sqlalchemy as db

In [75]:
c = configparser.ConfigParser()
c.read("config.ini")
zeit_key = c.get("Zeit", "key")
mapbox_key = c.get("Mapbox", "key")

#set the API, authenticate and check status
api = zeit.API()
api.set_token(zeit_key)
api.get_status()


'everything ok'

# Getting all the Data

In [3]:
def geocode(query):
    """ function to geocode places with the Mapbox API """
    endpoint = "mapbox.places"
    params = {"language": "de", "limit": 1, "autocomplete": False, 
    "access_token": mapbox_key,"types": "country,region,district,place,locality"}

    url = f"https://api.mapbox.com/geocoding/v5/{endpoint}/{query}.json"

    try:
        resp = requests.get(url = url, params=params).json()
        lon, lat = resp["features"][0]["center"]
        name = resp["features"][0]["place_name"]
        type = resp["features"][0]["place_type"]
        #get the short code of the country of the query
        if type[0] == "country":
            code = resp["features"][0]["properties"]["short_code"]
        else:
            for c in resp["features"][0]["context"]:
                if c["id"].startswith("country"):
                    code = c["short_code"]

        return {"lat": lat, "lon": lon, "name": name, "type": type, "country_code": code}
    except:
        return {"lat": None, "lon": None, "name": None, "type": None, "country_code": None}

In [ ]:
df = pd.DataFrame(columns = ["id", "keyword", "score", "lat", "lon", "name", "type", "country_code"])

for i in range(16):
    
    offset = i * 1000
    print(f"keywords {offset} - {offset+1000}")

    url = "http://api.zeit.de/keyword"
    params = {"limit": 1000, "offset": offset, "q": ""}
    headers = {"X-Authorization": zeit_key}

    resp = requests.get(url = url, params=params, headers = headers)
    search = zeit.Search("None", resp.json())

    liste = []
    for m in search.get_matches().values():
        m = m[1]
        if m["type"] == "location":
            liste.append(m)


    for o in liste:
        row = {}
        row["id"] = o["id"]
        row["keyword"] = o["value"]
        row["score"] = o["score"]

        geo = geocode(o["value"])
        row["lat"] = geo["lat"]
        row["lon"] = geo["lon"]
        row["name"] = geo["name"]
        row["type"] = geo["type"]
        row["country_code"] = geo["country_code"]
        
        df = df.append(row, ignore_index=True)

In [ ]:
converter_iso = {
'AF':'AFG',
'AX':'ALA',
'AL':'ALB',
'DZ':'DZA',
'AS':'ASM',
'AD':'AND',
'AO':'AGO',
'AI':'AIA',
'AQ':'ATA',
'AG':'ATG',
'AR':'ARG',
'AM':'ARM',
'AW':'ABW',
'AU':'AUS',
'AT':'AUT',
'AZ':'AZE',
'BS':'BHS',
'BH':'BHR',
'BD':'BGD',
'BB':'BRB',
'BY':'BLR',
'BE':'BEL',
'BZ':'BLZ',
'BJ':'BEN',
'BM':'BMU',
'BT':'BTN',
'BO':'BOL',
'BA':'BIH',
'BW':'BWA',
'BV':'BVT',
'BR':'BRA',
'IO':'IOT',
'BN':'BRN',
'BG':'BGR',
'BF':'BFA',
'BI':'BDI',
'KH':'KHM',
'CM':'CMR',
'CA':'CAN',
'CV':'CPV',
'KY':'CYM',
'CF':'CAF',
'TD':'TCD',
'CL':'CHL',
'CN':'CHN',
'CX':'CXR',
'CC':'CCK',
'CO':'COL',
'KM':'COM',
'CG':'COG',
'CD':'COD',
'CK':'COK',
'CR':'CRI',
'CI':'CIV',
'HR':'HRV',
'CU':'CUB',
'CY':'CYP',
'CZ':'CZE',
'DK':'DNK',
'DJ':'DJI',
'DM':'DMA',
'DO':'DOM',
'EC':'ECU',
'EG':'EGY',
'SV':'SLV',
'GQ':'GNQ',
'ER':'ERI',
'EE':'EST',
'ET':'ETH',
'FK':'FLK',
'FO':'FRO',
'FJ':'FJI',
'FI':'FIN',
'FR':'FRA',
'GF':'GUF',
'PF':'PYF',
'TF':'ATF',
'GA':'GAB',
'GM':'GMB',
'GE':'GEO',
'DE':'DEU',
'GH':'GHA',
'GI':'GIB',
'GR':'GRC',
'GL':'GRL',
'GD':'GRD',
'GP':'GLP',
'GU':'GUM',
'GT':'GTM',
'GG':'GGY',
'GN':'GIN',
'GW':'GNB',
'GY':'GUY',
'HT':'HTI',
'HM':'HMD',
'VA':'VAT',
'HN':'HND',
'HK':'HKG',
'HU':'HUN',
'IS':'ISL',
'IN':'IND',
'ID':'IDN',
'IR':'IRN',
'IQ':'IRQ',
'IE':'IRL',
'IM':'IMN',
'IL':'ISR',
'IT':'ITA',
'JM':'JAM',
'JP':'JPN',
'JE':'JEY',
'JO':'JOR',
'KZ':'KAZ',
'KE':'KEN',
'KI':'KIR',
'KP':'PRK',
'KR':'KOR',
'KW':'KWT',
'KG':'KGZ',
'LA':'LAO',
'LV':'LVA',
'LB':'LBN',
'LS':'LSO',
'LR':'LBR',
'LY':'LBY',
'LI':'LIE',
'LT':'LTU',
'LU':'LUX',
'MO':'MAC',
'MK':'MKD',
'MG':'MDG',
'MW':'MWI',
'MY':'MYS',
'MV':'MDV',
'ML':'MLI',
'MT':'MLT',
'MH':'MHL',
'MQ':'MTQ',
'MR':'MRT',
'MU':'MUS',
'YT':'MYT',
'MX':'MEX',
'FM':'FSM',
'MD':'MDA',
'MC':'MCO',
'MN':'MNG',
'ME':'MNE',
'MS':'MSR',
'MA':'MAR',
'MZ':'MOZ',
'MM':'MMR',
'NA':'NAM',
'NR':'NRU',
'NP':'NPL',
'NL':'NLD',
'AN':'ANT',
'NC':'NCL',
'NZ':'NZL',
'NI':'NIC',
'NE':'NER',
'NG':'NGA',
'NU':'NIU',
'NF':'NFK',
'MP':'MNP',
'NO':'NOR',
'OM':'OMN',
'PK':'PAK',
'PW':'PLW',
'PS':'PSE',
'PA':'PAN',
'PG':'PNG',
'PY':'PRY',
'PE':'PER',
'PH':'PHL',
'PN':'PCN',
'PL':'POL',
'PT':'PRT',
'PR':'PRI',
'QA':'QAT',
'RE':'REU',
'RO':'ROU',
'RU':'RUS',
'RW':'RWA',
'BL':'BLM',
'SH':'SHN',
'KN':'KNA',
'LC':'LCA',
'MF':'MAF',
'PM':'SPM',
'VC':'VCT',
'WS':'WSM',
'SM':'SMR',
'ST':'STP',
'SA':'SAU',
'SN':'SEN',
'RS':'SRB',
'SC':'SYC',
'SL':'SLE',
'SG':'SGP',
'SK':'SVK',
'SI':'SVN',
'SB':'SLB',
'SO':'SOM',
'ZA':'ZAF',
'GS':'SGS',
'ES':'ESP',
'LK':'LKA',
'SD':'SDN',
'SS':'SSD',
'SR':'SUR',
'SJ':'SJM',
'SZ':'SWZ',
'SE':'SWE',
'CH':'CHE',
'SY':'SYR',
'TW':'TWN',
'TJ':'TJK',
'TZ':'TZA',
'TH':'THA',
'TL':'TLS',
'TG':'TGO',
'TK':'TKL',
'TO':'TON',
'TT':'TTO',
'TN':'TUN',
'TR':'TUR',
'TM':'TKM',
'TC':'TCA',
'TV':'TUV',
'UG':'UGA',
'UA':'UKR',
'AE':'ARE',
'GB':'GBR',
'US':'USA',
'UM':'UMI',
'UY':'URY',
'UZ':'UZB',
'VU':'VUT',
'VE':'VEN',
'VN':'VNM',
'VG':'VGB',
'VI':'VIR',
'WF':'WLF',
'EH':'ESH',
'YE':'YEM',
"ZA":"ZAF",
'ZM':'ZMB',
'ZW':'ZWE',
}

converter_iso =  {k.lower(): v for k, v in converter_iso.items()}
converter_iso[None] = None

In [ ]:
df["iso_3"] = df.country_code.apply(lambda x: converter_iso[x])

In [ ]:
df.head()

In [ ]:
i = df.loc[dff.id == "china"].index.item()
df.loc[i] = ["china", "China", 98,35.4867029846329,101.901875103385 ,"Volksrepublik China", ['country'], 'cn', 'CHN']

i = df.loc[dff.id == "grossbritannien"].index.item()
df.loc[i] = ["grossbritannien", "Großbritannien", 98, 54.2379333607472, -2.36966957036279 ,"Vereinigtes Königreich", ['country'], 'gb', 'GBR']

i = df.loc[dff.id == "niger"].index.item()
df.loc[i] = ["niger", "Niger", 25, 17.4261488452015, 9.39764774206789 ,"Niger", ['country'], 'ne', 'NER']

In [ ]:
df.to_csv("../data/keywords.csv")
df.to_json("../data/keywords.json")

***

# Mapping

In [ ]:
df = pd.read_json("../data/keywords.json")

In [ ]:
df.shape

In [ ]:
def mapper(df):
    px.set_mapbox_access_token(mapbox_key)
    fig = px.scatter_mapbox(
        df, lat = "lat", lon = "lon", mapbox_style="carto-positron", custom_data=["keyword"],
        color = "score",size = "score", color_continuous_scale = px.colors.sequential.Sunsetdark
        )
    fig.update_mapboxes(
        center = dict(
            lon = 10,
            lat = 50,
        ),
        zoom = 2,
    )
    fig.update_traces(
        hovertemplate = "<b> %{customdata[0]} </b>: %{marker.size}" 
    )
    return fig


In [ ]:
fig = mapper(df)
fig.show()

In [ ]:
dff = df.copy()
countries = dff.explode("type")[dff.explode("type")["type"] == "country"]
countries = countries.drop_duplicates(subset = ["name"])

In [ ]:
fig = px.choropleth(
    countries, locations = "iso_3", color = "score", color_continuous_scale = px.colors.sequential.Reds_r,
    hover_name = "name"
    )


In [ ]:
fig.update_traces(
    marker_line_width = 0,
    showscale = False,
)
fig.update_layout(
    title = None,
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular',
        showocean = True, oceancolor = "#f4f4f4",
    ),
    showlegend = False,
    legend = dict(
        x = 0,
        y = 1
    ),
    paper_bgcolor = "#f4f4f4",
    plot_bgcolor = "#f4f4f4",

)



In [ ]:
de = df[df.country_code == "de"]

In [ ]:
figure = px.density_mapbox(de, lat = "lat", lon = "lon", z = "score", mapbox_style="carto-positron",color_continuous_scale = px.colors.sequential.Sunsetdark_r)
figure.show()

In [ ]:
f = open("../data/custom.geo.json")
geojson = json.load(f)

In [ ]:
px.choropleth_mapbox(countries, geojson = geojson, locations="iso_3", featureidkey = "properties.iso_a3", mapbox_style = "carto-positron", color = "score")